In [1]:
#normal tools:
from scipy import sparse
import numpy as np
import copy
import sys
sys.path.append("..")
import utils

#learning library:
import implicit

#skopt:
from skopt.space import Real, Integer
from skopt import Optimizer
from sklearn.externals.joblib import Parallel, delayed
from skopt.utils import use_named_args
import skopt




#load the 243-protein subset:
interaction_matrix = utils.load_subset()

#this performs multiple repeats of the test/train split, if desired:
def bootstrap(params, matrix, repeats):
    results = list()
    for _ in range(repeats):
        train, test = utils.train_test_split(interaction_matrix, 0.05)

        #train matrix goes to train the model
        pred_matrix = utils.train_implicit_bpr(params, train)
        #test matrix gets used to score predictions
        results.append(utils.evaluate_predictions(pred_matrix, test).mean())

    return np.mean(results)


####SKOPT:

#these are the hyperparameters and search spaces:
space = [Integer(1, 400, name='factors'),
        Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
        Real(10**-5, 10**0, "log-uniform", name='regularization'),
        Integer(1,20, name='iterations')]


#the objective function for skopt:
@use_named_args(space)
def score(**params):
    score = bootstrap(params, interaction_matrix, 1)
    return (score)

optimizer = Optimizer(dimensions=space,
                     random_state=1,
                     base_estimator='ET',
                     n_random_starts=20)



/Users/lmar3213/miniconda3/envs/lew_conda/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '
/Users/lmar3213/miniconda3/envs/lew_conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#6*6 = 36 iterations, with 20 being random.
for i in range(6):
    print(i)
    x = optimizer.ask(n_points=4)
    y = Parallel(n_jobs=4)(delayed(score)(v) for v in x)
    optimizer.tell(x,y)

result = skopt.utils.create_result(optimizer.Xi,
                                  optimizer.yi,
                                  optimizer.space,
                                  optimizer.rng,
                                  models=optimizer.models)



0
1
2


KeyboardInterrupt: 

In [ ]:
hi=result.space

In [ ]:
baaa = hi.dimensions[0]

In [ ]:
for j,k in zip(space, result.x_iters[np.argmin(result.func_vals)]):
    print(j.name, k)

In [ ]:

#write the data so we don't need to repeat it:
outfile = open('delete_me.dat', 'w')

outfile.write('Best parameters:\n')
for j,k in zip(space, result.x_iters[np.argmin(result.func_vals)]):
    outfile.write(str(j.name)+' '+str(k)+'\n')
outfile.write('Result: ')
outfile.write(str(result.fun))


outfile.write('\n\nAll:\n')
for j,k in zip(result.x_iters, result.func_vals):
    outfile.write(str(j)+' '+str(k)+'\n')
outfile.close()